# Data Analysis Nanodegree - Project 3 Code
**By: Juan Silva**

## Data Set

**Area Chosen:**
Ottawa, Ontario, Canada (Downtown)

**Open Street Map Link:**
http://www.openstreetmap.org/#map=13/45.4000/-75.6920

**File:**
ottawa.osm (83.3 MB)


In [ ]:
# IMPORTANT: MAKE SURE TO RUN THIS CELL FIRST BEFORE ANY CODE IN CELLS BELOW

#define some global variables required for execution
import xml.etree.cElementTree as ET
import pprint
import re
from collections import defaultdict
import codecs
import json
#File to be processed
filename = "ottawa.osm"
#Helper regex vars to check tags keys condition
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

## Initial Exploration

In [ ]:
#Loop all the tags using an event based approach (Good for large files that can't fit in memory)
tags = {}
for event, elem in ET.iterparse(filename):
    if elem.tag not in tags:
        tags[elem.tag] = 1
    else:
        tags[elem.tag] = tags[elem.tag] + 1
pprint.pprint(tags)

## Data Audit

#### Key Structure

In [ ]:
#This function evaluates each tag element and classifies it in three buckets
def key_type(element, keys):
    if element.tag == "tag":
        val = element.attrib["k"]
        if lower.match(val):
            keys["lower"] += 1
        elif lower_colon.match(val):
            keys["lower_colon"] += 1
        elif problemchars.match(val):
            keys["problemchars"] += 1
        else:
            keys["other"] += 1
    return keys

#loop all elements checking tag's keys by type and counting them
keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

keys

#### Street Naming and Postal Code Consistency

In [ ]:
#New entries to handle the french version of street names
expected_french = ["Rue", "Boulevard", "Chemin", "Promenade", "Way"]
street_type_french_re = re.compile(r'^\b\S+\.?', flags=re.IGNORECASE)

#English version with added entries
street_type_re = re.compile(r'\b\S+\.?$', flags=re.IGNORECASE)
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Crescent", "Private", "Way", "Circle", "Driveway", 
            "South", "North", "East", "West", "Terrace"]

#Check if a particular street has an unexpected name and log it, handling the french scenario
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            #Let's check the french version before logging it
            m_french = street_type_french_re.search(street_name)
            street_type_french = m_french.group()
            if street_type_french not in expected_french:
                street_types[street_type].add(street_name)

#Check if a particular element is a tag describing a street name
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

#Loop all elements and check those that are nodes or ways (which can have addresses)
street_types = defaultdict(set)
for _, elem in ET.iterparse(filename):
    if elem.tag == "node" or elem.tag == "way":
        for tag in elem.iter("tag"):
            if is_street_name(tag):
                audit_street_type(street_types, tag.attrib['v'])

pprint.pprint(dict(street_types))

In [ ]:
#Check postal codes
from pymongo import MongoClient
client = MongoClient('localhost:27017')
db = client.nanodegree

pprint.pprint([doc for doc in db.ottawa.aggregate([
        {"$match":{"address.postcode": {"$not": re.compile("^[A-Za-z]\d[A-Za-z] ?\d[A-Za-z]\d$")}}}, 
        {"$group":{"_id":"$address.postcode", "count":{"$sum":1}}}, 
        {"$sort":{"count":-1}}
    ])])


## Data Cleaning and Import

In [ ]:
#Attributes of the new "Created" object for nodes 
CREATED = [ "version", "changeset", "timestamp", "user", "uid"]

#The mapping dictionary for street abreviations and corresponding full words
mapping = { "Ave": "Avenue",
            "Blvd.": "Boulevard",
            "Blvd": "Boulevard",
            "Boul.": "Boulevard",
            "Dr.": "Drive",
            "Rd": "Road"
            }
#The mapping dictionary for postal codes that have incorrect entries
postalMappings = {
    'K2A OE8': 'K2A 0E8',
    'K2P 0P4,': 'K2P 0P4',
    'JOX 2W1': 'J0X 2W1',
    'K1N 7E7\u200e': 'K1N 7E7',
    'K1N 9N8': 'K1N 9N8'
}
#The mapping dictionary to determine if parking fee enrtries require payment
mappingPaidParking = {
    'Daily fee': 'yes',
    'no': 'no',
    'interval': 'yes',
    'yes': 'yes',
    'customer parking': 'no',
    'customers': 'no'
}


#Function used to fix street names as the JSON output is generated
def update_name(name, mapping):
    m = street_type_re.search(name)
    if m and m.group() in mapping:
        name = re.sub(r'\b\S+\.?$', mapping[m.group()],name)
    m_french = street_type_french_re.search(name)
    if m_french and m_french.group() in mapping:
        name = re.sub(r'^\b\S+\.?', mapping[m_french.group()],name)
    return name

#Function used to fix postal codes as the JSON output is generated
def update_code(code, mapping):
    if code in mapping:
        return mapping[code]
    else:
        return code

#Function used to generate a new field that defines if payment is required for a parking entry
def isPaidParking(val, mapping):
    if val in mapping:
        return mapping[val]
    else:
        return val


#Aux function to process a single xml element creating the JSON model
def shape_element(element, mapping, postalMappings, mappingPaidParking):
    node = {}
    if element.tag == "node" or element.tag == "way" :
        attrib = element.attrib
        #General Fields
        node["id"] = attrib["id"]
        if "visible" in attrib:
            node["visible"] = attrib["visible"]
        #Created Fields
        node["created"] = {}
        for item in CREATED:
            node["created"][item] = attrib[item]
        #Type tags
        node["type"] = element.tag
        if element.tag == "node":
            node["pos"] = [float(attrib["lat"]), float(attrib["lon"])]
        if element.tag == "way":
            node["node_refs"] = []
        #Process all tags, including address ones
        node["address"] = {}
        for child in element:
            if child.tag == "tag": 
                if child.attrib["k"] == "addr:housenumber":
                    node["address"]["housenumber"] = child.attrib["v"]
                elif child.attrib["k"] == "addr:postcode":
                    node["address"]["postcode"] = update_code(child.attrib["v"], postalMappings)
                elif child.attrib["k"] == "addr:street":
                    node["address"]["street"] = update_name(child.attrib["v"], mapping)
                else:
                    if child.attrib["k"] == "fee":
                        node["paid"] = isPaidParking(child.attrib["v"], mappingPaidParking)
                    node[child.attrib["k"]] = child.attrib["v"]
            if child.tag == "nd":
                node["node_refs"].append(child.attrib["ref"])
        if not node["address"]:
            del node["address"]
        return node
    else:
        return None

#Parse the input XML file and output the JSON file
file_out = "{0}.json".format(filename)
with codecs.open(file_out, "w") as fo:
    for _, element in ET.iterparse(filename):
        el = shape_element(element, mapping, postalMappings, mappingPaidParking)
        if el:
            fo.write(json.dumps(el) + "\n")

print "Finished"

After this step, we have a json file in disk which is clean, formatted and ready to be imported. In order to do that we ran the following command:
> mongoimport --host 127.0.0.1:27017 -d nanodegree -c ottawa  --file ottawa.osm.json

Then we check the database to see if it conforms to what we would expect:

In [ ]:
#Connect to the DB
from pymongo import MongoClient
client = MongoClient('localhost:27017')
db = client.nanodegree

print ("===== This is the total of Nodes + Ways listed above ====")
print db.ottawa.count(); 

print "==== This should match the general expected structure ===="
pprint.pprint(db.ottawa.find_one()) 

print "==== This should match the expected structure for a way ===="
pprint.pprint(db.ottawa.find({"type":"way"})[0]) 

print "==== The structure with one that has an address ===="
pprint.pprint(db.ottawa.find({"address": {"$exists": 1}})[0]) 

Once we have verified that the data was imported as expected. We are ready to do some analysis in MongoDB

## Data Analysis

In [ ]:
#Agregation, grouping by user and counting the elements (Output limmited to 100 for clarity in report)
pprint.pprint([doc for doc in db.ottawa.aggregate([
            {"$group": {"_id": "$created.user", "Contributed Items": {"$sum": 1}}},
            {"$project": {"_id": 0, "User": "$_id", "Contributed Items": 1}},
            {"$sort": {"Contributed Items": -1}},
            {"$limit": 100}
        ])
])

In [ ]:
#Agregation, Average contribution per user
pprint.pprint([doc for doc in db.ottawa.aggregate([
            {"$group": {"_id": "$created.user", "ContributedItems": {"$sum": 1}}},
            {"$group": {"_id": "User Average", "Contributed Items": {"$avg": "$ContributedItems"}}},
        ])
])

In [ ]:
#Aggregation grouping by amenity and counting entries
pprint.pprint([doc for doc in db.ottawa.aggregate([
            {"$match": {"amenity": {"$exists": 1}}},
            {"$group": {"_id": "$amenity", "Count": {"$sum": 1}}},
            {"$sort": {"Count": -1}}
        ])
])

In [ ]:
#Map Reduce job to identify all the unique keys across all Parking documents
map = "function() { if(this.amenity == 'parking'){ for (var key in this) { emit(key, null); } } }"
reduce = "function(key, stuff) { return null; }"
result = db.ottawa.map_reduce(map, reduce, "myresults")
for doc in result.find():
    print doc

In [ ]:
#Breakdown by kind of Fee related to parking
pprint.pprint([doc for doc in db.ottawa.aggregate([
                {"$match": {"amenity": "parking"}},
                {"$group": {"_id": "$fee", "Count": {"$sum": 1}}},
            ])])

In [ ]:
#Breakdown by kind of Access
pprint.pprint([doc for doc in db.ottawa.aggregate([
                {"$match": {"amenity": "parking"}},
                {"$group": {"_id": "$access", "Count": {"$sum": 1}}},
            ])])

In [ ]:
#Breakdown by type of parking
pprint.pprint([doc for doc in db.ottawa.aggregate([
                {"$match": {"amenity": "parking"}},
                {"$group": {"_id": "$parking", "Count": {"$sum": 1}}},
            ])])

In [ ]:
#Map Reduce job to identify all the unique keys across all Restaurant documents
map = "function() { if(this.amenity == 'restaurant'){ for (var key in this) { emit(key, null); } } }"
reduce = "function(key, stuff) { return null; }"
result = db.ottawa.map_reduce(map, reduce, "myresults")
for doc in result.find():
    print doc

In [ ]:
#Breakdown by type of Cuisine
pprint.pprint([doc for doc in db.ottawa.aggregate([
                {"$match": {"amenity": "restaurant"}},
                {"$group": {"_id": "$cuisine", "Count": {"$sum": 1}}},
                {"$sort": {"Count": -1}}
            ])])

In [ ]:
#Breakdown by Delivery availability
pprint.pprint([doc for doc in db.ottawa.aggregate([
                {"$match": {"amenity": "restaurant"}},
                {"$group": {"_id": "$delivery", "Count": {"$sum": 1}}},
                {"$sort": {"Count": -1}}
            ])])

In [ ]:
#Breakdown by Wheelchair Accessibility
pprint.pprint([doc for doc in db.ottawa.aggregate([
                {"$match": {"amenity": "restaurant"}},
                {"$group": {"_id": "$wheelchair", "Count": {"$sum": 1}}},
                {"$sort": {"Count": -1}}
            ])])